# Decoding DLVR data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xdf_interface as xif

## Import Data

In [ ]:
target_framerate = 250;
timeframe_start = 1.5

In [ ]:
# Common path for all files to cut down on repetition
path = ""
# File names. testfiles is only used for final evaluation and not loaded until after the training
trainfiles = np.array(['data_1.xdf','data_2.xdf'])
validfiles = np.array(['data_4.xdf','data_5.xdf'])
testfiles = np.array(['data_6.xdf', 'data_7.xdf'])

In [ ]:
X,y = xif.bdonline_extract(path,trainfiles,timeframe_start,target_framerate)
Xv, yv = xif.bdonline_extract(path, validfiles, timeframe_start, target_framerate)

## Decode Data
This uses the braindecode tutorial code, set to use the Deep4 network instead of ShallowNet, to decode the data.

In [ ]:
# braindecode version: 0.4.84
from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(X, y=y)
valid_set = SignalAndTarget(Xv, y=yv)

In [ ]:
from braindecode.models.deep4 import Deep4Net
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = 64
input_time_length = 625
# final_conv_length = auto ensures we only get a single output in the time dimension
# cropsize / receptive field of the network is: final_conv_length*81 + 360
# final conv_length = 2 translates into a receptive fields of 522 samples, roughly 2 seconds of data at 250 Hz
model = Deep4Net(in_chans=in_chans, n_classes=n_classes,
                        input_time_length= input_time_length,
                        final_conv_length=2, batch_norm=True)
if cuda:
    model.cuda()


In [ ]:
from braindecode.torch_ext.optimizers import AdamW
from braindecode.torch_ext.losses import log_categorical_crossentropy
import torch.nn.functional as F

optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=1*0.001) # these are good values for the deep model
#optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1, cropped = True)

In [ ]:
model.fit(train_set.X, train_set.y, epochs=50, batch_size=32, scheduler='cosine',
          input_time_length=input_time_length, #remember_best_column='valid_misclass', 
         validation_data=(valid_set.X, valid_set.y),)

## Results

In [ ]:
model.epochs_df

In [ ]:
model.epochs_df[["train_loss", "valid_loss"]].plot(logy=True)

In [ ]:
model.epochs_df[["train_misclass", "valid_misclass"]].plot()

## Testing

In [ ]:
Xt, yt = mif.bdonline_extract(path, testfiles, timeframe_start, target_framerate)

In [ ]:
from braindecode.datautil.signal_target import SignalAndTarget

test_set = SignalAndTarget(Xt, y=yt)

In [ ]:
model.evaluate(test_set.X, test_set.y)

## Saving

In [ ]:
import torch
# braindecode online requires the filename "deep_4_params"
torch.save(model.network.state_dict(), "\\deep_4_params")